In [1]:
import pandas as pd
from io import StringIO
import requests
from tqdm.notebook import tqdm

## récupère la liste des circonscriptions

In [2]:
circonscriptions_url = 'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2022/referenceLG/listeregdptcom.xml'

In [3]:
departements = (
    pd
    .read_xml(
        circonscriptions_url,
        xpath='.//Departement'
    )
)

departements

,CodDpt,CodMinDpt,CodDpt3Car,NumOrdDpt,LibDpt,Communes
0,ZA,971,971,97,Guadeloupe,\n
1,ZB,972,972,98,Martinique,\n
2,ZC,973,973,99,Guyane,\n
3,ZD,974,974,100,La Réunion,\n
4,ZM,976,976,101,Mayotte,\n
...,...,...,...,...,...,...
102,ZP,987,987,103,Polynésie française,\n
103,ZS,975,975,104,Saint-Pierre-et-Miquelon,\n
104,ZW,986,986,106,Wallis et Futuna,\n
105,ZX,977,977,107,Saint-Martin/Saint-Barthélemy,\n


In [4]:
def get_circonscription(CodMinDpt):
    return (
        pd
        .read_xml(
            circonscriptions_url,
            xpath = f'.//Departement[./CodMinDpt = "{CodMinDpt}"]//Commune',
        )
        .assign(
            CodMinDpt = CodMinDpt
        )
    )

communes = pd.concat([ get_circonscription(CodMinDpt) for CodMinDpt in tqdm(departements.CodMinDpt.to_list()) ])

communes

100%|██████████████████████████████████████████████████████████████████████████████| 107/107 [06:32<00:00,  3.67s/it]


,CodSubCom,LibSubCom,CodCirLg,LibFraSubCom,CodMinDpt
0,101,Les Abymes,1,Les Abymes - 1ère circonscription,971
1,102,Anse-Bertrand,2,Anse-Bertrand - 2ème circonscription,971
2,103,Baie-Mahault,3,Baie-Mahault - 3ème circonscription,971
3,104,Baillif,4,Baillif - 4ème circonscription,971
4,105,Basse-Terre,4,Basse-Terre - 4ème circonscription,971
...,...,...,...,...,...
205,229,Zurich,6,Zurich - 6ème circonscription,99
206,231,Taipei,11,Taipei - 11ème circonscription,99
207,233,Nour-Soultan,11,Nour-Soultan - 11ème circonscription,99
208,234,Monterrey,2,Monterrey - 2ème circonscription,99


In [5]:
circonscriptions = (
    communes
    .groupby(['CodMinDpt', 'CodCirLg'])
    .agg({
        'CodSubCom' : lambda x: len(x.unique())
    })
    .reset_index()
    .join(
        departements.set_index('CodMinDpt'), on='CodMinDpt'
    )
)

circonscriptions

,CodMinDpt,CodCirLg,CodSubCom,CodDpt,CodDpt3Car,NumOrdDpt,LibDpt,Communes
0,01,1,82,01,001,1,Ain,\n
1,01,2,58,01,001,1,Ain,\n
2,01,3,64,01,001,1,Ain,\n
3,01,4,89,01,001,1,Ain,\n
4,01,5,102,01,001,1,Ain,\n
...,...,...,...,...,...,...,...,...
572,99,7,22,ZZ,099,108,Français établis hors de France,\n
573,99,8,12,ZZ,099,108,Français établis hors de France,\n
574,99,9,18,ZZ,099,108,Français établis hors de France,\n
575,99,10,47,ZZ,099,108,Français établis hors de France,\n


In [6]:
circonscriptions.to_csv('lg2022_circonscriptions.csv', index=False)

## premier tour

### récupère la liste des résultats

In [7]:
def get_resultats_t1(circonscription):
    dept = circonscription[:3]
    circ = circonscription

    url = f'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT1/{dept}/{circonscription}.xml'
    
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath='.//Candidat'
            )
            .assign(CodCirc=circonscription)
        )
    except:
        print(f'error: {circonscription}')
        df = None
        
    
    return df
    
    
resultats_2022_t1 = pd.concat(
    [
        get_resultats_t1(circonscription)
        for circonscription in
            tqdm(
                (
                    circonscriptions.CodDpt3Car
                    + circonscriptions.CodCirLg.astype(str).str.pad(2, fillchar='0')
                ).to_list()
            )
    ]
)
resultats_2022_t1

100%|██████████████████████████████████████████████████████████████████████████████| 577/577 [01:03<00:00,  9.15it/s]


,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc
0,1,GUILLERMIN,Vincent,M.,ENS,Ensemble ! (Majorité présidentielle),8071,"19,30","9,36",non,00101
1,2,LAHY,Éric,M.,DXG,Divers extrême gauche,391,"0,94","0,45",non,00101
2,3,BRETON,Xavier,M.,LR,Les Républicains,10599,"25,35","12,30",Bal.,00101
3,4,PIROUX GIANNOTTI,Brigitte,Mme,RN,Rassemblement National,8971,"21,46","10,41",non,00101
4,5,MENDES,Michael,M.,DSV,Droite souverainiste,641,"1,53","0,74",non,00101
...,...,...,...,...,...,...,...,...,...,...,...
3,4,TAPAYEVA,Tamila,Mme,DSV,Droite souverainiste,481,"1,74","0,49",non,09911
4,5,MARTIN,Catya,Mme,LR,Les Républicains,1760,"6,37","1,78",non,09911
5,6,VIDAL,Dominique,Mme,NUP,Nouvelle union populaire écologique et sociale,6849,"24,79","6,93",Bal.,09911
6,7,BURLOTTE,Olivier,M.,RN,Rassemblement National,742,"2,69","0,75",non,09911


### récupére le nombre d'inscrits pour chaque circonscription

In [8]:
def get_inscrits_t1(circonscription):
    url = f'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT1/{circonscription[:3]}/{circonscription}.xml'
    
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath='.//Inscrits'
            )
            .assign(CodCirc=circonscription)
        )
    except:
        print(f'error: {circonscription}')
        df = None
        
    
    return df

inscrits_2022_t1 = pd.concat([
    get_inscrits_t1(circonscription)
        for circonscription in
            tqdm(
                (
                    circonscriptions.CodDpt3Car
                    + circonscriptions.CodCirLg.astype(str).str.pad(2, fillchar='0')
                ).to_list()
            )
])

100%|██████████████████████████████████████████████████████████████████████████████| 577/577 [01:01<00:00,  9.37it/s]


In [9]:
(
    inscrits_2022_t1
    .join(
        circonscriptions
        .assign(
            CodCirc0 = lambda df: (
                df.CodDpt3Car
                + df.CodCirLg.astype(str).str.pad(2, fillchar='0')
            ),
            CodCirc1 = lambda df: (
                df.CodDpt3Car
                + df.CodCirLg.astype(str).str.pad(2, fillchar='0')
            ),
            CodCirc2 = lambda df: (
                df.CodDpt
                + df.CodCirLg.astype(str).str.pad(3, fillchar='0')
            )
        )
        .set_index('CodCirc0')[['CodCirc1', 'CodCirc2']],
        on = 'CodCirc'
    )
    .set_index('CodCirc2')
    [['Nombre']]
    .rename({ 'Nombre': 'inscrits' }, axis=1)
    .to_csv('lg2022/t1_inscrits.csv')
)

### crée un fichier pour l'ensemble des résultats

In [10]:
resultats_2022_t1_2 = (
    resultats_2022_t1
    .join(
        circonscriptions
        .assign(
            CodCirc0 = lambda df: (
                df.CodDpt3Car
                + df.CodCirLg.astype(str).str.pad(2, fillchar='0')
            ),
            CodCirc1 = lambda df: (
                df.CodDpt3Car
                + df.CodCirLg.astype(str).str.pad(2, fillchar='0')
            ),
            CodCirc2 = lambda df: (
                df.CodDpt
                + df.CodCirLg.astype(str).str.pad(3, fillchar='0')
            )
        )
        .set_index('CodCirc0')[['CodCirc1', 'CodCirc2']],
        on = 'CodCirc'
    )
)

(
    resultats_2022_t1_2
    .to_csv(
        'lg2022/t1_resultats.csv',
        index=False
    )
)

### crée un fichier unique pour chaque circonscription

In [11]:
for circo in list(resultats_2022_t1_2.CodCirc2.unique()):
    (
        resultats_2022_t1_2
        .query('CodCirc2 == @circo')
        .to_csv(f'lg2022/t1/{circo}.csv', index=False)
    )

### crée un tableau synthétique de présence des nuances

In [12]:
(
    resultats_2022_t1_2
    .rename(columns=
        {
            'CodNua': 'Code nuance',
            'NomPsn': 'Nom du candidat'
        }
    )
    .pivot_table(
        index = 'CodCirc2',
        columns = 'Code nuance',
        values = 'Nom du candidat',
        aggfunc= lambda x : len(x) >= 1
    )
    .to_csv('lg2022_t1_nuances.csv')
)

## second tour

### récupère les résultats

In [13]:
def get_resultats_t2(circonscription):
    dept = circonscription[:3]
    circ = circonscription
    
    url = f'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2022/resultatsT2/{dept}/{circ}.xml'
    
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath='.//Tour[./NumTour = "2"]//Candidat'
            )
            .assign(CodCirc=circonscription)
        )
    except:
        print(f'error: {circonscription}')
        df = None
        
    
    return df
    
    
resultats_2022_t2 = pd.concat([
    get_resultats_t2(circonscription)
    for circonscription in
        tqdm(
            (
                circonscriptions.CodDpt3Car
                + circonscriptions.CodCirLg.astype(str).str.pad(2, fillchar='0')
            ).to_list()
        )
])

resultats_2022_t2

 43%|█████████████████████████████████▊                                            | 250/577 [00:28<01:21,  3.99it/s]

error: 05303


 68%|█████████████████████████████████████████████████████▏                        | 393/577 [00:43<00:20,  9.13it/s]

error: 07506


 70%|██████████████████████████████████████████████████████▎                       | 402/577 [00:44<00:20,  8.64it/s]

error: 07516
error: 07517


 89%|█████████████████████████████████████████████████████████████████████▌        | 515/577 [00:56<00:06, 10.01it/s]

error: 09307


100%|██████████████████████████████████████████████████████████████████████████████| 577/577 [01:02<00:00,  9.16it/s]


,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc
0,3,BRETON,Xavier,M.,LR,Les Républicains,24408,"63,22","28,32",oui,00101
1,8,GUÉRAUD,Sébastien,M.,NUP,Nouvelle union populaire écologique et sociale,14203,"36,78","16,48",non,00101
0,1,LAPRAY,Lumir,Mme,NUP,Nouvelle union populaire écologique et sociale,17824,"41,66","17,83",non,00102
1,2,DAUBIÉ,Romain,M.,ENS,Ensemble ! (Majorité présidentielle),24960,"58,34","24,97",oui,00102
0,3,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),18398,"58,72","22,38",oui,00103
...,...,...,...,...,...,...,...,...,...,...,...
1,3,BEN CHEÏKH,Karim,M.,NUP,Nouvelle union populaire écologique et sociale,11348,"54,07","9,41",oui,09909
0,4,LAKRAFI,Amélia,Mme,ENS,Ensemble ! (Majorité présidentielle),13048,"63,58","12,49",oui,09910
1,9,MOUSSA,Chantal,Mme,NUP,Nouvelle union populaire écologique et sociale,7474,"36,42","7,16",non,09910
0,1,GENETET,Anne,Mme,ENS,Ensemble ! (Majorité présidentielle),16537,"61,73","16,73",oui,09911


### isole les circonscriptions résolues au premier tour

In [14]:
resultats_2022_t1_direct = (
    resultats_2022_t1_2
    #.query('~CodCirc2.isin(@resultats_2022_t2_2.CodCirc2)')
    .query('Elu == "oui"')
)

resultats_2022_t1_direct

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc,CodCirc1,CodCirc2
5,6,FAVENNEC,Yannick,M.,ENS,Ensemble ! (Majorité présidentielle),19187,"57,13","26,49",oui,05303,05303,53003
3,4,CHIKIROU,Sophia,Mme,NUP,Nouvelle union populaire écologique et sociale,24155,"53,74","29,85",oui,07506,07506,75006
0,1,LEGRAIN,Sarah,Mme,NUP,Nouvelle union populaire écologique et sociale,20829,"56,51","27,73",oui,07516,07516,75016
1,2,OBONO,Danièle,Mme,NUP,Nouvelle union populaire écologique et sociale,16162,"57,07","26,01",oui,07517,07517,75017
10,11,CORBIÈRE,Alexis,M.,NUP,Nouvelle union populaire écologique et sociale,22718,"62,94","28,16",oui,09307,09307,93007


### crée un fichier pour l'ensemble des résultats

In [15]:
resultats_2022_t2_2 = (
    resultats_2022_t2
    .join(
        circonscriptions
        .assign(
            CodCirc0 = lambda df: (
                df.CodDpt3Car
                + df.CodCirLg.astype(str).str.pad(2, fillchar='0')
            ),
            CodCirc1 = lambda df: (
                df.CodDpt3Car
                + df.CodCirLg.astype(str).str.pad(2, fillchar='0')
            ),
            CodCirc2 = lambda df: (
                df.CodDpt
                + df.CodCirLg.astype(str).str.pad(3, fillchar='0')
            )
        )
        .set_index('CodCirc0')[['CodCirc1', 'CodCirc2']],
        on = 'CodCirc'
    )
)

(
    resultats_2022_t2_2
    .to_csv(
        'lg2022/t2_resultats.csv',
        index=False
    )
)

### crée un fichier unique pour chaque circonscription

In [16]:
for circo in list(resultats_2022_t2_2.CodCirc2.unique()):
    (
        resultats_2022_t2_2
        .query('CodCirc2 == @circo')
        .to_csv(f'lg2022/t2/{circo}.csv', index=False)
    )

### crée un tableau synthétique de présence des nuances

In [17]:
resultats_2022_t2_nuances_pos = (
    pd
    .concat([
        resultats_2022_t2_2
        .pivot_table(
            index = 'CodCirc2',
            columns = 'CodNua',
            values = 'NbVoix',
            aggfunc = 'sum'
        ),
        resultats_2022_t1_direct
        .pivot_table(
            index = 'CodCirc2',
            columns = 'CodNua',
            values = 'NbVoix',
            aggfunc = 'sum'
        )
    ])
    .sort_index()
    .rank(axis=1, ascending=False)
    .fillna(0)
    .astype(int)
)

resultats_2022_t2_nuances_pos

CodNua,DIV,DSV,DVC,DVD,DVG,DXG,ENS,LR,NUP,REG,RN,UDI
CodCirc2,,,,,,,,,,,,
01001,0,0,0,0,0,0,0,1,2,0,0,0
01002,0,0,0,0,0,0,1,0,2,0,0,0
01003,0,0,0,0,0,0,1,0,2,0,0,0
01004,0,0,0,0,0,0,0,0,2,0,1,0
01005,0,0,0,1,0,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ007,0,0,0,0,0,0,1,0,2,0,0,0
ZZ008,0,0,0,0,0,0,2,0,0,0,0,1
ZZ009,0,0,0,0,0,0,2,0,1,0,0,0


In [18]:
resultats_2022_t2_nuances_pos.to_csv('lg2022_t2_nuances_pos.csv')